In [ ]:
import os
import pandas as pd
import random
import requests

In [ ]:
# Define the paths to your CSV files
good_csv_path = 'goodOutput.csv'
bad_csv_path = 'badOutput.csv'

# Define the output directory structure
output_base_dir = 'output'
output_dirs = ['test/good', 'test/bad', 'train/good', 'train/bad']

# Create the output directories
for output_dir in output_dirs:
    os.makedirs(os.path.join(output_base_dir, output_dir), exist_ok=True)

# Load the CSV files into pandas dataframes
good_df = pd.read_csv(good_csv_path)
bad_df = pd.read_csv(bad_csv_path)

# Define the number of random rows to select for testing
num_test_samples = 500

# Randomly select rows for testing and training from both good and bad dataframes
good_test_samples = random.sample(good_df.index.tolist(), num_test_samples)
bad_test_samples = random.sample(bad_df.index.tolist(), num_test_samples)

# Download and save the test set images
for index in good_test_samples:
    url = good_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'test/good', f'good_test_{index}.jpg'), 'wb') as f:
            f.write(response.content)

for index in bad_test_samples:
    url = bad_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'test/bad', f'bad_test_{index}.jpg'), 'wb') as f:
            f.write(response.content)

# Create lists of rows for the training set
good_train_samples = [index for index in good_df.index if index not in good_test_samples]
bad_train_samples = [index for index in bad_df.index if index not in bad_test_samples]

# Calculate the number of rows to select for training (minimum of both files)
num_train_samples = min(len(good_train_samples), len(bad_train_samples))

# Randomly select the same number of rows from both good and bad dataframes for training
train_samples_good = random.sample(good_train_samples, num_train_samples)
train_samples_bad = random.sample(bad_train_samples, num_train_samples)



# Download and save the training set images
for index in train_samples_good:
    url = good_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'train/good', f'good_train_{index}.jpg'), 'wb') as f:
            f.write(response.content)

for index in train_samples_bad:
    url = bad_df.at[index, 'Thumbnail']
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(output_base_dir, 'train/bad', f'bad_train_{index}.jpg'), 'wb') as f:
            f.write(response.content)

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow.keras.preprocessing.image import ImageDataGenerator

We shall use the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

To know more about ImageDataGenerator class, visit https://keras.io/preprocessing/image/#imagedatagenerator-class

In [ ]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    'output/train',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    'output/test',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary')

Found 2850 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Our model will have 3 Convolution2D layers. You can increse or decrease as per your needs.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),

    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 128)       0

We shall use Adam optimizer with a learning rate of 0.001 (hyperparameter). We choose 'binary_crossentropy' loss as our model is a binary calssifier (i.e, we have only 2 classes)

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

At the end of each epoch we can check if the model has reached the required accuracy and terminate the training.

In [ ]:
DESIRED_ACCURACY = 0.90

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}) :
        if(logs.get('acc') is not None and logs.get('acc') >= DESIRED_ACCURACY) :
            print('\nReached 95% accuracy so cancelling training!')
            self.model.stop_training = True

callbacks = myCallback()

To know more about fit_generator visit https://keras.io/models/model/#fit_generator

In [ ]:
model.fit_generator(
    train_datagenerator,
    epochs=100,
    validation_data = test_datagenerator,
    callbacks = [callbacks]
    )

Epoch 1/100
 1/90 [..............................] - ETA: 6s - loss: 1.4143e-07 - accuracy: 1.0000

<ipython-input-9-9318e07318eb>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


90/90 [==============================] - 7s 82ms/step - loss: 1.1952e-07 - accuracy: 1.0000 - val_loss: 3.8581 - val_accuracy: 0.7810
Epoch 2/100
90/90 [==============================] - 5s 58ms/step - loss: 1.1066e-07 - accuracy: 1.0000 - val_loss: 3.8717 - val_accuracy: 0.7810
Epoch 3/100
90/90 [==============================] - 6s 72ms/step - loss: 1.0713e-07 - accuracy: 1.0000 - val_loss: 3.8764 - val_accuracy: 0.7800
Epoch 4/100
90/90 [==============================] - 5s 59ms/step - loss: 1.0645e-07 - accuracy: 1.0000 - val_loss: 3.8878 - val_accuracy: 0.7800
Epoch 5/100
90/90 [==============================] - 7s 72ms/step - loss: 9.9588e-08 - accuracy: 1.0000 - val_loss: 3.9026 - val_accuracy: 0.7800
Epoch 6/100
90/90 [==============================] - 5s 58ms/step - loss: 9.8383e-08 - accuracy: 1.0000 - val_loss: 3.9109 - val_accuracy: 0.7810
Epoch 7/100
90/90 [==============================] - 6s 69ms/step - loss: 8.8553e-08 - accuracy: 1.0000 - val_loss: 3.9221 - val_accurac

In [ ]:
model.save('mymodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/mymodel.h5" "/content/drive/My Drive/Latestmodel.h5"

In [ ]:
model = tf.keras.models.load_model('mymodel.h5')


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
path = 'download1.jpg'
img = image.load_img(path, target_size=(128, 128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images)
print(classes[0][0])
if classes[0]<0.5:
    print("Given image is bad")
else:
    print("Given image is good")

1/1 [==============================] - 0s 79ms/step
0.0
Given image is bad


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Load your pre-trained model
model = tf.keras.models.load_model('mymodel.h5')

# Define the directory path
image_directory = 'shiba'  # Replace with the actual path to your image directory

# Iterate through the image files in the directory
for filename in os.listdir(image_directory):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct the full path to the image
        image_path = os.path.join(image_directory, filename)

        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(128, 128))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)

        # Make predictions
        classes = model.predict(x)
        probability = classes[0][0]

        if probability < 0.5:
            print(f"Image {filename} - Given image is bad")
        else:
            print(f"Image {filename} - Given image is good")


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 76ms/step
Image 5.jpg - Given image is good
1/1 [==============================] - 0s 17ms/step
Image 1.jpg - Given image is good
1/1 [==============================] - 0s 17ms/step
Image 6.jpg - Given image is bad
1/1 [==============================] - 0s 20ms/step
Image 3.jpg - Given image is good
1/1 [==============================] - 0s 19ms/step
Image 4.jpg - Given image is bad
1/1 [==============================] - 0s 18ms/step
Image 2.jpg - Given image is good
